In [3]:
import cv2
import numpy as np

# tuplify
def tup(point):
    return (point[0], point[1]);

# returns true if the two boxes overlap
def overlap(source, target):
    # unpack points
    tl1, br1 = source;
    tl2, br2 = target;

    # checks
    if (tl1[0] >= br2[0] or tl2[0] >= br1[0]):
        return False;
    if (tl1[1] >= br2[1] or tl2[1] >= br1[1]):
        return False;
    return True;

# returns all overlapping boxes
def getAllOverlaps(boxes, bounds, index):
    overlaps = [];
    for a in range(len(boxes)):
        if a != index:
            if overlap(bounds, boxes[a]):
                overlaps.append(a);
    return overlaps;

img = cv2.imread("Images/1.png")
orig = np.copy(img);
blue, green, red = cv2.split(img)

def medianCanny(img, thresh1, thresh2):
    median = np.median(img)
    img = cv2.Canny(img, int(thresh1 * median), int(thresh2 * median))
    return img

blue_edges = medianCanny(blue, 0, 1)
green_edges = medianCanny(green, 0, 1)
red_edges = medianCanny(red, 0, 1)

edges = blue_edges | green_edges | red_edges

# I'm using OpenCV 3.4. This returns (contours, hierarchy) in OpenCV 2 and 4
contours,hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_SIMPLE)

# go through the contours and save the box edges
boxes = []; # each element is [[top-left], [bottom-right]];
hierarchy = hierarchy[0]
for component in zip(contours, hierarchy):
    currentContour = component[0]
    currentHierarchy = component[1]
    x,y,w,h = cv2.boundingRect(currentContour)
    if currentHierarchy[3] < 0:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1)
        boxes.append([[x,y], [x+w, y+h]]);

# filter out excessively large boxes
filtered = [];
max_area = 30000;
for box in boxes:
    w = box[1][0] - box[0][0];
    h = box[1][1] - box[0][1];
    if w*h < max_area:
        filtered.append(box);
boxes = filtered;

# go through the boxes and start merging
merge_margin = 15;

# this is gonna take a long time
finished = False;
highlight = [[0,0], [1,1]];
points = [[[0,0]]];
while not finished:
    # set end con
    finished = True;

    # check progress
    print("Len Boxes: " + str(len(boxes)));

    # draw boxes # comment this section out to run faster
    copy = np.copy(orig);
    for box in boxes:
        cv2.rectangle(copy, tup(box[0]), tup(box[1]), (0,200,0), 1);
    cv2.rectangle(copy, tup(highlight[0]), tup(highlight[1]), (0,0,255), 2);
    for point in points:
        point = point[0];
        cv2.circle(copy, tup(point), 4, (255,0,0), -1);
    cv2.imshow("Copy", copy);
    key = cv2.waitKey(1);
    if key == ord('q'):
        break;

    # loop through boxes
    index = len(boxes) - 1;
    while index >= 0:
        # grab current box
        curr = boxes[index];

        # add margin
        tl = curr[0][:];
        br = curr[1][:];
        tl[0] -= merge_margin;
        tl[1] -= merge_margin;
        br[0] += merge_margin;
        br[1] += merge_margin;

        # get matching boxes
        overlaps = getAllOverlaps(boxes, [tl, br], index);
        
        # check if empty
        if len(overlaps) > 0:
            # combine boxes
            # convert to a contour
            con = [];
            overlaps.append(index);
            for ind in overlaps:
                tl, br = boxes[ind];
                con.append([tl]);
                con.append([br]);
            con = np.array(con);

            # get bounding rect
            x,y,w,h = cv2.boundingRect(con);

            # stop growing
            w -= 1;
            h -= 1;
            merged = [[x,y], [x+w, y+h]];

            # highlights
            highlight = merged[:];
            points = con;

            # remove boxes from list
            overlaps.sort(reverse = True);
            for ind in overlaps:
                del boxes[ind];
            boxes.append(merged);

            # set flag
            finished = False;
            break;

        # increment
        index -= 1;
cv2.destroyAllWindows();

# show final
copy = np.copy(orig);
for box in boxes:
    cv2.rectangle(copy, tup(box[0]), tup(box[1]), (0,200,0), 1);
cv2.imshow("Final", copy);
cv2.waitKey(0);

Len Boxes: 360
Len Boxes: 352
Len Boxes: 351
Len Boxes: 350
Len Boxes: 332
Len Boxes: 326
Len Boxes: 302
Len Boxes: 272
Len Boxes: 190
Len Boxes: 148
Len Boxes: 111
Len Boxes: 79
Len Boxes: 72
Len Boxes: 69
Len Boxes: 67
Len Boxes: 63
Len Boxes: 36
Len Boxes: 20
Len Boxes: 16
Len Boxes: 13
Len Boxes: 12
Len Boxes: 8
Len Boxes: 6


In [1]:
from pytesseract import *
import cv2

In [2]:
images = cv2.imread("1.png")
rgb = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
results = pytesseract.image_to_data(rgb, output_type=Output.DICT)
print(results)

{'level': [1, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5], 'page_num': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [3]:
for i in range(0, len(results["text"])):
      
    # We can then extract the bounding box coordinates
    # of the text region from  the current result
    # x = results["left"][i]
    # y = results["top"][i]
    # w = results["width"][i]
    # h = results["height"][i]
      
    # We will also extract the OCR text itself along
    # with the confidence of the text localization
    text = results["text"][i]
    conf = int(results["conf"][i])
    

text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text: OMTPECE
conf: 8
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text: (OMTECH
conf: 0
text: CHEMICAL
conf: 95
text: INDUSTRIES
conf: 96
text: PVT.
conf: 96
text: LTD.
conf: 96
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text:  
conf: 95
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text:  
conf: 95
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text:  
conf: 95
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text:  
conf: 95
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text:  
conf: 95
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text:  
conf: 95
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text:  
conf: 95
text: 
conf: -1
text: 
conf: -1
text: 
conf: -1
text: PaySlip
conf: 85
text: 62
conf: 53
text: Payslip
conf: 63
text: for
conf: 96
text: the
conf: 96
text: month
conf: 92
text: :
conf: 92
text: December
conf: 96
text: 2018
conf: 76
text: Branel
conf: 38
text: ANDHERI
conf: 28
text: 
conf: -1
text: 

In [11]:
import cv2
import numpy as np

# Read image
image = cv2.imread('temp/bounding/bounding_roi_0.png')
# cv2.imshow()
cv2.imshow("txt", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Convert image to grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Use canny edge detection
edges = cv2.Canny(gray,50,150,apertureSize=3)

# Apply HoughLinesP method to
# to directly obtain line end points
lines_list =[]
lines = cv2.HoughLinesP(
			edges, # Input edge image
			1, # Distance resolution in pixels
			np.pi/180, # Angle resolution in radians
			threshold=100, # Min number of votes for valid line
			minLineLength=5, # Min allowed length of line
			maxLineGap=10 # Max allowed gap between line for joining them
			)

# Iterate over points
for points in lines:
	# Extracted points nested in the list
	x1,y1,x2,y2=points[0]
	# Draw the lines joing the points
	# On the original image
if (y1-y2)==0:
	cv2.line(image,(x1,y1),(x2,y2),(0,255,0),2)
	# Maintain a simples lookup list for points
	lines_list.append([(x1,y1),(x2,y2)])
	
# Save the result image
cv2.imwrite('detectedLines.png',image)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 35)

In [12]:
import cv2

In [18]:
img = cv2.imread("Images/2.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

for i, contour in enumerate(contours):
    if i==0:
        continue

    epsilon = 0.01*cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    cv2.drawContours(img, contour, 0, (0,0,0), 4)
    x,y,w,h = cv2.boundingRect(approx)

    x_mid = int(x+(w/3))
    y_mid = int(y+(h/1.5))

    coords = (x_mid, y_mid)
    color = (0,0,0)
    # font = cv2.FONT_HERSHEY_DUPLEX

    # if len(approx)==4:
        # cv2.putText(img, "Quadrilateral", coords, font, 1, color, 1)

cv2.imshow("shapes", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
import img_2_txt as cvrt

txt = cvrt.txt_from_img(img)
print(txt)

Af? Situry WP

tn Words t Rupets Sixty Seven Thousand Three Hundred Seventy Seven only:

Notes Site thts @ 2 tormttepanttted stuttrrgnt t doesnot nsed mtty etanature,,

ge ste

TEL HOME FINANCE LiMttED

_Devsu om rte Wir er 2bHt
Eenpttyes Netre PRASHANT RANSTTRUMAR SABUIMALA — Enptoyee Gode | O1ABSYS
Datignation 7 SALES MANAGER Laeetien, 3 SURAT
Departrtant + SALES Jen Date UBIO TE
‘PAN Nurtibtr ASUPSasbOS PF anetunt No, .
‘Brant SURAT MING ROAD ‘Butig Att, No, DBMTSOLOSLAS
Berk Name ‘HOFe ornde + MANAGER
Ne, Dionne Days, 30.50 ‘Leane Teen 2 8888
‘UANND : wwe 70080
ESCNO .

EARWINGS if DEDUCTIONS
BASIC SALARY 2300400 [PROFESSION TAK 208.00
HOUSE RENT ALLOWANCE 1202.00
CONVEYANCE ALLOWANCE 1B7aD0
EDUCATION tor0D
LEAVE TRAVEL ALLOWANCE zebtee
AMEDITAL ALLOWANCE 1236.68
‘VEAL ALLUWANGE 2184,00
SUPPLEMENTARY ALLOWANCE baattod
Bross Amsunt 67877.06 | ‘Teta Deduttion 2be08
Ret Satsry if arent




In [22]:

 
# Read image to convert image to string
img = cv2.imread('Images/1.png')
 
# Resize the image if required
height, width, channel = img.shape
# images = cv2.resize(img, (width//2, height//2))
 
# Convert to grayscale image
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
# Converting grey image to binary image by Thresholding
thresh_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
 
# configuring parameters for tesseract
custom_config = r'--oem 3 --psm 6'
 
# Converting image to text with pytesseract
ocr_output = pytesseract.image_to_string(thresh_img)
# Print output text from OCR
print(ocr_output)
 
# Writing OCR output to a text file
with open('python_ocr_output.txt', 'w') as f:
    f.write(ocr_output)
 
# Display image
cv2.imshow('Gray image', images)
cv2.waitKey(0)

OME ECE ‘OMTECH CHEMICAL INDUSTRIES PVT. LTD.

PayStip e Payalip for the month : December 2018 Branel ANDHERI

EmpCode:  TOM02, Name Mr. VIRENDRA VITHAL KOLBEKAR Basle: 12760.00

Grade A Dopartment : EXPORT Designation: EXPORT EXECUTIVE

ESIC Ne. VAN No. : 100409628642 Division :

Joining Dt. 1042018

Days Pald = 28.00 Days Present: 23.00 WOMPdOn: — 4.0070.00 LWPiAbsent: 0.007300

si 100 cL: 00 PL 0.00 co+ico 0.00 10.00

Bal Su: 0.00 Bal. cL: 9.90 Bal. PL: 4.00 Bal. CO: 0.00

Earnings & Reimbursements Gross Amt Actual Amt [Deductions & Recoveries Gross Amt Actual Amt

‘Eamed Basic 12780.00 +4816.00" | PROV, FUND 1599.00 1382.00

HRA 7275.00 6571.00 | P.Tax 0.00 200.00

Transport Allowance 6275.00 5868.00

Total Earnings 2630.00 28765.00 | Total Deductions: 1582.00
Nat Pa 2273.00

‘Net Pay: Rupees Twenty Two Thousand One Hundred Seventy Three Only
Bank Name: BANK OF INDIA. Bank Ale No. 0161010000531 1
Checked by: Approved by: Received by:



-1

In [26]:
import cv2  # import OpenCV library
 
# Read image for contour detection
input_image = cv2.imread("Images/1.png")
 
# Make a copy to draw bounding box
input_image_cpy = input_image.copy()
 
# Show input image in OpenCV
cv2.imshow('Input image', input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
# Convert input image to grayscale
gray_img = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
 
# Show grey image in OpenCV
cv2.imshow('Grey image', gray_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
threshold_value = gray_img[216, 402]
print(threshold_value)
 
# Convert the grayscale image to binary (image binarization opencv python)
ret, binary_img = cv2.threshold(gray_img, threshold_value, 255, cv2.THRESH_BINARY)
 
# Show binary image in OpenCV
cv2.imshow('Binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
# Invert image
inverted_binary_img = ~ binary_img
 
# Show binary image in OpenCV
cv2.imshow('Inverted image', inverted_binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
# Detect contours
# hierarchy variable contains information about the relationship between each contours
contours_list, hierarchy = cv2.findContours(inverted_binary_img,
                                       cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE) # Find contours
 
# Draw first contour
first_contour = 0
second_contour = 1
 
contour1 = cv2.drawContours(input_image, contours_list, first_contour,(255,0,255),3)
cv2.imshow('First detected contour', contour1)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
# Draw a bounding box around the first contour
x, y, w, h = cv2.boundingRect(contours_list[first_contour])
cv2.rectangle(contour1,(x,y), (x+w,y+h), (0,0,255), 5)
cv2.imshow('First contour with bounding box', contour1)
cv2.waitKey(0)
cv2.destroyAllWindows()
 
# Draw a bounding box around all detected contours
for c in contours_list:
    x, y, w, h = cv2.boundingRect(c)
 
    # Make sure contour area is large enough
    if (cv2.contourArea(c)) > 1000:
        cv2.rectangle(input_image_cpy, (x, y), (x + w, y + h), (0, 0, 255), 5)
 
cv2.imshow('All contours with bounding box', input_image_cpy)
cv2.waitKey(0)
cv2.destroyAllWindows()

243


In [31]:
import pytesseract
from pytesseract import Output
# Import OpenCV library
import cv2
 
import csv
 
# Read image to extract text from image
img = cv2.imread('Images/2.png')
# Resize the image if required
height, width, channel = img.shape
# img = cv2.resize(img, (width//2, height//2))
 
# Convert image to grey scale
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
# Converting grey image to binary image by Thresholding
thresh_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
 
# configuring parameters for tesseract
custom_config = r'--oem 3 --psm 6'
 
# Get all OCR output information from pytesseract
ocr_output_details = pytesseract.image_to_data(thresh_img, output_type = Output.DICT, config=custom_config, lang='eng')
# Total bounding boxes
n_boxes = len(ocr_output_details['level'])
print("No. of boxes: ", n_boxes)
 
# Extract and draw rectangles for all bounding boxes
for i in range(n_boxes):
    (x, y, w, h) = (ocr_output_details['left'][i], ocr_output_details['top'][i], ocr_output_details['width'][i], ocr_output_details['height'][i])
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
 
# Print OCR Output kesys
print(ocr_output_details.keys())
 
# Show output image with bounding boxes
cv2.imshow('img', img)
cv2.waitKey(0)

No. of boxes:  160
dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


-1

In [33]:
print(ocr_output_details["text"])
print(ocr_output_details["conf"])

['', '', '', '', 'PP', 'Sutury', '(KP', '', '.', 'Hf', 'i', '', 'FL', 'HOME', 'FINANCE', 'LIMITED', '', '1IFL', 'PAYSLIP', 'FOR', 'THE', 'MONTH', 'OF', 'OCT', '2018', '', '-', '—————', '', 'Employee', 'Name', 'PRASHANT', 'RANJITKUMAR', 'SABUWALA', 'Employee', 'Code.', '146235', '', 'Designation', 'SALES', 'MANAGER', 'Location', 'SURAT', '', 'Department', 'SALES', 'doin', 'Date', '26/07/2016', '', 'PAN', 'Number', 'ASUPS4860J', 'PF', 'account', 'No.', '0', '', 'Branch', 'SURAT-RING', 'ROAD', 'Bank', 'Acc.', 'No.', '02511050095925', '', 'Bank', 'Name', 'HDFG', 'Grade', 'MANAGER', '', 'No.', 'Of', 'Working', 'Days', '30.50', 'Leave', 'Taken', '00.00', '', 'UAN', 'NO', 'Lwe', '00.50', '', 'ESIC', 'NO', '', '[aan', 'rections', '', 'BASIC', 'SALARY', '24004.00', '|', 'PROFESSION', 'TAX', '200.00', '', 'HOUSE', 'RENT', 'ALLOWANCE', '12002.00', '', 'CONVEYANCE', 'ALLOWANCE', '1874.00', '', 'EDUCATION', '197.00', '', 'px)', 'LEAVE', 'TRAVEL', 'ALLOWANCE', '2000.00', '', 'MEDICAL', 'ALLOWANCE', 